## Time setup and predictions for different theories

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 140
mpl.rcParams['figure.dpi'] = 140
import numpy as np
# our own modules
from lace_manager.data import data_MPGADGET
from lace.emulator import gp_emulator
from lace.emulator import p1d_archive
from lace_manager.likelihood import likelihood
from lace_manager.likelihood import full_theory
from lace_manager.likelihood import lya_theory
import time

### Set up mock data

Begin by picking a simulation to use as mock data, and creating a corresponding data object

In [2]:
# specify simulation to use to generate synthetic data
test_sim_label="diff_z"
if type(test_sim_label)==int:
    drop_sim_number=test_sim_label
    print('will drop sim number {} from emulator'.format(drop_sim_number))
else:
    drop_sim_number=None

In [3]:
# specify simulation suite and P1D mesurements
basedir="/lace/emulator/sim_suites/Australia20/"
skewers_label='Ns500_wM0.05'
p1d_label=None
z_max=4.5
data=data_MPGADGET.P1D_MPGADGET(basedir=basedir,
                                skewers_label=skewers_label,
                                sim_label=test_sim_label,
                                zmax=z_max,pivot_scalar=0.05)

H0 = 6.7000E+01, Omega_b h^2 = 2.2000E-02, Omega_c h^2 = 1.2000E-01, Omega_k = -5.5511E-17, Omega_nu h^2 = 0.0000E+00, A_s = 2.0061E-09, n_s = 9.6757E-01, alpha_s = 0.0000E+00


### Emulator and training set

Create a set of training data to train an emulator

In [4]:
# Set up emulator training set
z_max=4
archive=p1d_archive.archiveP1D(basedir=basedir,drop_sim_number=drop_sim_number,
                            drop_tau_rescalings=True,z_max=z_max,
                            drop_temp_rescalings=True,skewers_label=skewers_label)

In [5]:
## Build emulator
paramList=['mF', 'sigT_Mpc', 'gamma', 'kF_Mpc', 'Delta2_p', 'n_p']
# specify k range
kmax_Mpc=8
emu=gp_emulator.GPEmulator(basedir,p1d_label,skewers_label,z_max=z_max,
                                verbose=False,paramList=paramList,train=True,
                                asymmetric_kernel=True,rbf_only=True,
                                emu_type="k_bin",passarchive=archive,
                                kmax_Mpc=kmax_Mpc)

 /Users/font/opt/anaconda3/envs/lya_cosmo/lib/python3.8/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 270 points
Optimised


### Setup theories

In [6]:
t0=time.time()
theory_fFF=full_theory.FullTheory(zs=data.z,emulator=emu,theta_MC=True,use_compression=False,use_camb_fz=False)
t1=time.time()
print("Theory setup in {0:.5f} seconds".format(t1-t0))

Theory setup in 0.03140 seconds


In [7]:
t0=time.time()
theory_fFT=full_theory.FullTheory(zs=data.z,emulator=emu,theta_MC=True,use_compression=False,use_camb_fz=True)
t1=time.time()
print("Theory setup in {0:.5f} seconds".format(t1-t0))

Theory setup in 0.00632 seconds


In [8]:
t0=time.time()
theory_fTF=full_theory.FullTheory(zs=data.z,emulator=emu,theta_MC=True,use_compression=True,use_camb_fz=False)
t1=time.time()
print("Theory setup in {0:.5f} seconds".format(t1-t0))

Theory setup in 4.58524 seconds


In [9]:
t0=time.time()
theory_fTT=full_theory.FullTheory(zs=data.z,emulator=emu,theta_MC=True,use_compression=True,use_camb_fz=True)
t1=time.time()
print("Theory setup in {0:.5f} seconds".format(t1-t0))

Theory setup in 0.69507 seconds


In [10]:
t0=time.time()
theory_lF=lya_theory.LyaTheory(zs=data.z,emulator=emu,use_camb_fz=False,verbose=False)
t1=time.time()
print("Theory setup in {0:.5f} seconds".format(t1-t0))

Theory setup in 5.03240 seconds


In [11]:
t0=time.time()
theory_lT=lya_theory.LyaTheory(zs=data.z,emulator=emu,use_camb_fz=True)
t1=time.time()
print("Theory setup in {0:.5f} seconds".format(t1-t0))

Theory setup in 0.75186 seconds


### Time theory predictions

In [12]:
for theory in [theory_fFF,theory_fFT,theory_fTF,theory_fTT]:
    free_param_limits=[[0.0099,0.011],[1.1e-09, 3.19e-09], [0.89, 1.05],[-0.2, 0.2]]
    free_param_names=["cosmomc_theta","As","ns","ln_tau_0"]
    like=likelihood.Likelihood(data=data,theory=theory,
                            free_param_names=free_param_names,
                            free_param_limits=free_param_limits,
                            prior_Gauss_rms=None,include_CMB=False)
    t0=time.time()
    chi2=like.get_chi2(values=[0.46,0.49,0.51,0.51])
    t1=time.time()
    print("chi2={0:.5f} , computed in {1:.5f} seconds".format(chi2,t1-t0))

chi2=75.61440 , computed in 0.41166 seconds
chi2=75.61440 , computed in 0.44279 seconds
chi2=75.25236 , computed in 0.80593 seconds
chi2=75.25254 , computed in 0.43663 seconds


In [13]:
for theory in [theory_lF,theory_lT]:
    free_param_names=["Delta2_star","f_star","ln_tau_0"]
    free_param_limits=[[0.24, 0.47], [-2.352, -2.25], [-0.2, 0.2]]
    like=likelihood.Likelihood(data=data,theory=theory,
                            free_param_names=free_param_names,
                            free_param_limits=free_param_limits,
                            prior_Gauss_rms=None,include_CMB=False)
    t0=time.time()
    chi2=like.get_chi2()
    t1=time.time()
    print("chi2={0:.5f} , computed in {1:.5f} seconds".format(chi2,t1-t0))

chi2=18.81179 , computed in 0.03318 seconds
chi2=18.81179 , computed in 0.02909 seconds
